In [1]:
# Core data wrangling
import pandas as pd
import numpy as np

# Reading parquet files
import pyarrow.parquet as pq
import duckdb  # query parquet files directly with SQL

# Working with dates and times
from datetime import datetime, timedelta

# Static plots
import matplotlib.pyplot as plt
import seaborn as sns

# Interactive plots and maps
import plotly.express as px
import plotly.graph_objects as go
import folium  # mapping
from folium.plugins import HeatMap

# Geospatial data
import geopandas as gpd
from shapely.geometry import Point, Polygon

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Time Series / Forecasting
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm  # auto_arima

# Advanced models
import xgboost as xgb

from tqdm import tqdm
import missingno as msno



import streamlit as st

The Point of this battle simulation is to see if I can accurately predict which pokemon will win a battle. Becasue I have limited data I will make a few assumptions/exceptions.
- First, instead of using the correct damage formula I will use Damage = (max(attack, spattack) aka power) * (Attack / Defense) * Type Multiplier * Randomness
- 

In [8]:
poke = pd.read_csv('pokemon_dataset_with_classes.csv')
poke

,dex_number,name,type_01,type_02,ability_01,ability_02,hidden_ability,egg_group_01,egg_group_02,is_legendary,...,attack,defense,sp_attack,sp_defense,speed,BST,type_combo,cluster,cluster2,class
0,#0001,Bulbasaur,Grass,Poison,Overgrow,NaN,Chlorophyll,Monster,Grass,False,...,49,49,65,65,45,318,Grass/Poison,1,1,Underpowered
1,#0002,Ivysaur,Grass,Poison,Overgrow,NaN,Chlorophyll,Monster,Grass,False,...,62,63,80,80,60,405,Grass/Poison,3,1,Underpowered
2,#0003,Venusaur,Grass,Poison,Overgrow,NaN,Chlorophyll,Monster,Grass,False,...,82,83,100,100,80,525,Grass/Poison,0,0,Tank
3,#0004,Charmander,Fire,NaN,Blaze,NaN,Solar Power,Monster,Dragon,False,...,52,43,60,50,65,309,Fire/None,1,1,Underpowered
4,#0005,Charmeleon,Fire,NaN,Blaze,NaN,Solar Power,Monster,Dragon,False,...,64,58,80,65,80,405,Fire/None,3,3,Fast Sweeper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,#1021,Raging Bolt,Electric,Dragon,Protosynthesis,NaN,NaN,NaN,NaN,False,...,73,91,137,89,75,590,Electric/Dragon,0,0,Tank
1021,#1022,Iron Boulder,Rock,Psychic,Quark,NaN,NaN,NaN,NaN,False,...,120,80,68,108,124,590,Rock/Psychic,0,0,Tank
1022,#1023,Iron Crown,Steel,Psychic,Quark,NaN,NaN,NaN,NaN,False,...,72,100,122,108,98,590,Steel/Psychic,0,0,Tank
1023,#1024,Terapagos,Normal,NaN,Tera,NaN,NaN,NaN,NaN,True,...,65,85,65,85,60,450,Normal/None,2,2,balanced


In [13]:
import random

# Type effectiveness function
def type_effectiveness(attacker_type, defender_type1, defender_type2=None):
    """Returns type multiplier for attacker's type against defender's types."""
    effectiveness_chart = {
    'Normal': {'Rock': 0.5, 'Ghost': 0, 'Steel': 0.5},
    'Fire': {'Fire': 0.5, 'Water': 0.5, 'Grass': 2, 'Ice': 2, 'Bug': 2, 'Rock': 0.5, 'Dragon': 0.5, 'Steel': 2},
    'Water': {'Fire': 2, 'Water': 0.5, 'Grass': 0.5, 'Ground': 2, 'Rock': 2, 'Dragon': 0.5},
    'Electric': {'Water': 2, 'Electric': 0.5, 'Grass': 0.5, 'Ground': 0, 'Flying': 2, 'Dragon': 0.5},
    'Grass': {'Fire': 0.5, 'Water': 2, 'Grass': 0.5, 'Poison': 0.5, 'Ground': 2, 'Flying': 0.5, 'Bug': 0.5, 'Rock': 2, 'Dragon': 0.5, 'Steel': 0.5},
    'Ice': {'Fire': 0.5, 'Water': 0.5, 'Grass': 2, 'Ice': 0.5, 'Ground': 2, 'Flying': 2, 'Dragon': 2, 'Steel': 0.5},
    'Fighting': {'Normal': 2, 'Ice': 2, 'Poison': 0.5, 'Flying': 0.5, 'Psychic': 0.5, 'Bug': 0.5, 'Rock': 2, 'Ghost': 0, 'Dark': 2, 'Steel': 2, 'Fairy': 0.5},
    'Poison': {'Grass': 2, 'Poison': 0.5, 'Ground': 0.5, 'Rock': 0.5, 'Ghost': 0.5, 'Steel': 0, 'Fairy': 2},
    'Ground': {'Fire': 2, 'Electric': 2, 'Grass': 0.5, 'Poison': 2, 'Flying': 0, 'Bug': 0.5, 'Rock': 2, 'Steel': 2},
    'Flying': {'Electric': 0.5, 'Grass': 2, 'Fighting': 2, 'Bug': 2, 'Rock': 0.5, 'Steel': 0.5},
    'Psychic': {'Fighting': 2, 'Poison': 2, 'Psychic': 0.5, 'Dark': 0, 'Steel': 0.5},
    'Bug': {'Fire': 0.5, 'Grass': 2, 'Fighting': 0.5, 'Poison': 0.5, 'Flying': 0.5, 'Psychic': 2, 'Ghost': 0.5, 'Dark': 2, 'Steel': 0.5, 'Fairy': 0.5},
    'Rock': {'Fire': 2, 'Ice': 2, 'Fighting': 0.5, 'Ground': 0.5, 'Flying': 2, 'Bug': 2, 'Steel': 0.5},
    'Ghost': {'Normal': 0, 'Psychic': 2, 'Ghost': 2, 'Dark': 0.5},
    'Dragon': {'Dragon': 2, 'Steel': 0.5, 'Fairy': 0},
    'Dark': {'Fighting': 0.5, 'Psychic': 2, 'Ghost': 2, 'Dark': 0.5, 'Fairy': 0.5},
    'Steel': {'Fire': 0.5, 'Water': 0.5, 'Electric': 0.5, 'Ice': 2, 'Rock': 2, 'Steel': 0.5, 'Fairy': 2},
    'Fairy': {'Fire': 0.5, 'Fighting': 2, 'Poison': 0.5, 'Dragon': 2, 'Dark': 2, 'Steel': 0.5}
    }

    mult1 = effectiveness_chart.get(attacker_type, {}).get(defender_type1, 1)
    mult2 = 1
    if defender_type2:
        mult2 = effectiveness_chart.get(attacker_type, {}).get(defender_type2, 1)
    return mult1 * mult2


In [27]:
# Simplified damage formula
def calculate_damage(attacker, defender):
    """
    attacker, defender: dicts with keys
    'Name', 'HP', 'Attack', 'Defense', 'SpAtk', 'SpDef', 'Speed', 'Type1', 'Type2'
    """
    # Power based on offensive stats
    power = 15
    
    # Use Attack / Defense ratio
    attack_stat = attacker['attack']
    defense_stat = defender['defense']
    
    # Type multiplier
    type_mult = type_effectiveness(attacker['type_01'], defender['type_01'], defender.get('type_02'))
    
    # Random factor for variability
    rand = random.uniform(0.7, 1.0)
    
    # Damage formula
    damage = power * (attack_stat / defense_stat) * type_mult * rand
    return damage


In [28]:
# Turn-based battle simulator
def simulate_battle(poke1, poke2, verbose=True):
    """
    Simulate a battle between two Pokémon until one's HP <= 0.
    Returns the winner's name.
    """
    # Initialize HP
    hp1 = poke1['hp']
    hp2 = poke2['hp']
    
    # Determine who goes first based on Speed
    if poke1['speed'] >= poke2['speed']:
        first, second = poke1, poke2
        hp_first, hp_second = hp1, hp2
    else:
        first, second = poke2, poke1
        hp_first, hp_second = hp2, hp1
    
    turn = 1
    while hp_first > 0 and hp_second > 0:
        # First Pokémon attacks
        dmg = calculate_damage(first, second)
        hp_second -= dmg
        if verbose:
            print(f"Turn {turn}: {first['name']} hits {second['name']} for {dmg:.1f} damage! HP left: {max(hp_second,0):.1f}")
        if hp_second <= 0:
            winner = first['name']
            break
        
        # Second Pokémon attacks
        dmg = calculate_damage(second, first)
        hp_first -= dmg
        if verbose:
            print(f"Turn {turn}: {second['name']} hits {first['name']} for {dmg:.1f} damage! HP left: {max(hp_first,0):.1f}")
        if hp_first <= 0:
            winner = second['name']
            break
        
        turn += 1
    
    if verbose:
        print(f"\nWinner: {winner}")
    return winner

In [30]:
# Example Pokémon
poke1 = poke.loc[poke['name']=='Mudkip'].to_dict('records')[0]
poke2 = poke.loc[poke['name']=='Pidgey'].to_dict('records')[0]

# Simulate battle
simulate_battle(poke1, poke2)


Turn 1: Pidgey hits Mudkip for 12.3 damage! HP left: 37.7
Turn 1: Mudkip hits Pidgey for 22.6 damage! HP left: 17.4
Turn 2: Pidgey hits Mudkip for 9.9 damage! HP left: 27.8
Turn 2: Mudkip hits Pidgey for 20.6 damage! HP left: 0.0

Winner: Mudkip


'Mudkip'

In [20]:
poke[poke['name'] == 'Dragonite']

,dex_number,name,type_01,type_02,ability_01,ability_02,hidden_ability,egg_group_01,egg_group_02,is_legendary,...,attack,defense,sp_attack,sp_defense,speed,BST,type_combo,cluster,cluster2,class
148,#0149,Dragonite,Dragon,Flying,Inner,NaN,Multiscale,Water 1,Dragon,False,...,134,95,100,100,80,600,Dragon/Flying,0,0,Tank
